In [ ]:
%load_ext autoreload
%autoreload 2

from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import holoviews as hv
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
# file path to data and configs

#prefix = "/home/radioglaciology/thomas/radar_data/20240510-bench-decay/20240510_234434"

#prefix = "/home/radioglaciology/thomas/radar_data/20240510-bench-decay/20240510_234556" # 20 us rx time
#prefix = "/home/radioglaciology/thomas/radar_data/20240510-bench-decay/20240510_234720" # 60 us rx time
prefix = "/home/radioglaciology/thomas/radar_data/20240510-bench-decay/20240510_234720" # 100 us rx time

# resave data as zarr for dask processing
zarr_path = pr.save_radar_data_to_zarr(prefix)

# open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})

### View raw pulse in time domain to check for clipping

In [ ]:
single_pulse_raw = raw.radar_data[{'pulse_idx': 0}].compute()
plot1 = np.real(single_pulse_raw).hvplot.line(x='fast_time', color='red', label="real") * np.imag(single_pulse_raw).hvplot.line(x='fast_time', color="blue", label="imag")

plot1 = plot1.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

In [ ]:
plot_objs = []

for pulse_idx in np.arange(4):
    single_pulse_raw = raw.radar_data[{'pulse_idx': pulse_idx}].compute()
    plot_objs.append(np.real(single_pulse_raw).hvplot.line(x='fast_time', label=f"pulse_idx {pulse_idx}"))

plot1 = hv.Overlay(plot_objs).opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
plot1

In [ ]:
raw.config['GENERATE']['chirp_length']

In [ ]:
radar_data_chirp = raw.radar_data.where(raw.fast_time <= raw.config['GENERATE']['chirp_length'], drop=True)
(20*np.log10(np.sqrt((np.abs(radar_data_chirp)**2).mean(dim='sample_idx')))).hvplot.scatter(x="pulse_idx").opts(ylabel="RX signal mean amplitude [dB]")

In [ ]:
#plots = []
p = (20*np.log10(np.sqrt((np.abs(radar_data_chirp)**2).mean(dim='sample_idx')))).hvplot.scatter(x="pulse_idx", label="100 us RX").opts(ylabel="RX signal mean amplitude [dB]")
plots.append(p)

In [ ]:
hv.Overlay(plots)